# 드라이브 연결 및 모듈 로드

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import re
import glob

# 데이터 로드

In [ ]:
eat_2018 = pd.read_csv("/content/drive/MyDrive/Team Space/data/raw_data/즉석섭취식품_2018_2020.csv", encoding = "utf-8")
eat_2021 = pd.read_csv("/content/drive/MyDrive/Team Space/data/raw_data/즉석섭취식품_2021_2022.csv", encoding = "utf-8")
eat_2018.shape, eat_2021.shape

((38, 122), (34, 34))

In [ ]:
cook_2018 = pd.read_csv("/content/drive/MyDrive/Team Space/data/raw_data/즉석조리식품_2018_2020.csv", encoding = "utf-8")
cook_2021 = pd.read_csv("/content/drive/MyDrive/Team Space/data/raw_data/즉석조리식품_2021_2022.csv", encoding = "utf-8")
cook_2018.shape, cook_2021.shape

((38, 242), (34, 130))

In [ ]:
fresh_2018 = pd.read_csv("/content/drive/MyDrive/Team Space/data/raw_data/신선편의식품_2018_2020.csv", encoding = "utf-8")
fresh_2021 = pd.read_csv("/content/drive/MyDrive/Team Space/data/raw_data/신선편의식품_2021_2022.csv", encoding = "utf-8")
fresh_2018.shape, fresh_2021.shape

((38, 50), (34, 50))

# 컬럼 설정 리스트

In [ ]:
# 재배열 위한 리스트
new_col_list2018 = ["year", "소계", "수도권", "충청권", "호남권", "대경권", "동남권", "강원권", "1인", "2인", "3인", "4인", "5인이상", "남성", "여성", "20대", "30대", "40대", "50대",
                "60대이상", "200만원미만", "200~300만원미만", "300~400만원미만", "400~500만원미만", "500~600만원미만", "600만원이상"]

In [ ]:
# 재배열 위한 리스트
new_col_list2021 = ['year', '소계', '수도권', '충청권', '호남권', '대경권', '동남권', '강원권', '1인', '2인', '3인', '4인이상', '남성', '여성', '20대', '30대', '40대', '50대', '60대이상',
                    '200만원미만', '200~400만원미만', '400~600만원미만','600만원이상']

In [ ]:
# 즉시섭취식품 맵핑 dict
eat_dict = {
    "샐러드" : "즉석섭취반찬류",
    "도시락" : "즉석섭취식사류",
    "김밥류" : "즉석섭취식사류",
    "샌드위치/햄버거류" : "즉석섭취섭취류",
    "반찬류" : "즉석섭취반찬류"
}

In [ ]:
# 즉시조리식품 맵핑 dict
# 특이사항 한번 체크 -> 양념류 주석에 없는데 파일에 존재
cook_dict = {
    "밥류" : "즉석밥류",
    "면류" : "즉석떡볶이/면류",
    "국류" : "즉석국류/찌개/탕류",
    "찌개/탕류" : "즉석국류/찌개/탕류",
    "죽/스프류" : "죽/스프류",
    "육류" : "육류",
    "수산물" : "수산물",
    "만두/피자류" : "만두/피자류",
    "소스류" : "소스/양념류",
    "앙념류" : "소스/양념"
}

In [ ]:
# 신선편의식품 맵핑 dict
fresh_dict = {
    "신선편의채소" : "신선편의채소",
    "신선편의과일" : "신선편의과일"
}

# 데이터 표쥰화 함수

In [ ]:
# 컬럼 2개 병합하고 나누는 함수
def sum_col(x, y):
    col = round((x.astype(float) + y.astype(float)) / 2, 1)
    return col

In [ ]:
def preprocessing2018(raw_df, check_dict):
    pattern = re.compile(r"\S")
    new_col_list2021 = ['항목', '소계', '수도권', '충청권', '호남권', '대경권', '동남권', '강원권', '1인', '2인', '3인', '4인이상', '남성', '여성', '20대', '30대', '40대', '50대', '60대이상',
                    '200만원미만', '200~400만원미만', '400~600만원미만','600만원이상']

    raw_df_index = raw_df["특성별(2)"]                                                                                 # 인덱스 분리
    year = [raw_df.columns.tolist()[i].split(".")[0] for i in range(len(raw_df.columns))][2:]                          # 연도 구분 분리
    year.append("year")                                                                                                # 개수 맞추기 위한 append
    raw_df_columns = raw_df.iloc[0]                                                                                    # 매핑하기 위해 분리
    raw_df_columns = raw_df_columns.map(check_dict) + "_" + raw_df.iloc[2]                                             # 매핑한 변수랑 부가 변수랑 형식 맞추기
    raw_df_columns.fillna("", inplace = True)                                                                          # 밑 줄 전처리 하기 위한 nan값 채우기
    raw_df_columns = raw_df_columns.apply(lambda x: "".join(pattern.findall(x.split("(")[0])))                         # 각 값 끝에 (%) or (점) 삭제
    raw_df.drop(["특성별(1)", "특성별(2)"], axis = 1, inplace = True)                                                  # 필요 없는 컬럼 삭제
    raw_df.drop([0, 1, 2], axis = 0, inplace = True)                                                                   # 필요 없는 컬럼 삭제
    raw_df.columns = raw_df_columns[2:]                                                                                # 컬럼 맞추기
    raw_df["항목"] = raw_df_index[3:].str.replace(" ", "", regex = True)                                               # 인덱스 띄어쓰기 없애기
    raw_df = raw_df.T                                                                                                  # 연도 추가하기 위해 행렬변환
    raw_df["year"] = year                                                                                              # 연도 추가
    new_col = raw_df.iloc[-1, :].values
    raw_df.columns = new_col
    raw_df = raw_df.iloc[:-1, :]
    raw_df = raw_df[new_col_list2018]                                                                                  # 순서 재배치
    new_year = raw_df["year"].values
    raw_df = raw_df.reset_index()
    raw_df.index = new_year
    raw_df.drop("year", axis = 1, inplace = True)
    raw_df = raw_df.rename(columns = {"index": "항목"})

    raw_df["4인이상"] = sum_col(raw_df["4인"], raw_df["5인이상"])
    raw_df["200~400만원미만"] = sum_col(raw_df["200~300만원미만"], raw_df["300~400만원미만"])
    raw_df["400~600만원미만"]= sum_col(raw_df["400~500만원미만"], raw_df["500~600만원미만"])
    raw_df.drop(["4인", "5인이상", "200~300만원미만", "300~400만원미만", "400~500만원미만", "500~600만원미만"], axis = 1, inplace = True)

    return raw_df[new_col_list2021]

In [ ]:
def preprocessing2021(raw_df):
    pattern = re.compile(r"\S")
    raw_df_index = raw_df["특성별(2)"]
    year = [raw_df.columns.tolist()[i].split(".")[0] for i in range(len(raw_df.columns))][2:]
    year.append("year")
    raw_df_columns = raw_df.iloc[0] + "_" + raw_df.iloc[2].str.replace("구입", "")
    raw_df_columns = raw_df_columns.apply(lambda x: "".join(pattern.findall(x.split("(")[0])))
    raw_df.drop(["특성별(1)", "특성별(2)"], axis = 1, inplace = True)
    raw_df.drop([0, 1, 2], axis = 0, inplace = True)
    raw_df.columns = raw_df_columns[2:]
    raw_df["항목"] = raw_df_index[3:].str.replace(" ", "", regex = True)
    raw_df = raw_df.T
    raw_df["year"] = year
    new_col = raw_df.iloc[-1, :].values
    raw_df.columns = new_col
    raw_df = raw_df.iloc[:-1, :]
    raw_df = raw_df[new_col_list2021]
    new_year = raw_df["year"].values
    raw_df = raw_df.reset_index()
    raw_df.index = new_year
    raw_df.drop("year", axis = 1, inplace = True)
    raw_df = raw_df.rename(columns = {"index": "항목"})
    return raw_df

# 데이터 만들기

## 즉석섭취식품

In [ ]:
# 즉석섭취식품 / 즉석조리식품 / 신선편의식품 전처리 후 병합
eat_2018 = preprocessing2018(eat_2018, eat_dict)
eat_2021 = preprocessing2021(eat_2021)
cook_2018 = preprocessing2018(cook_2018, cook_dict)
cook_2021 = preprocessing2021(cook_2021)
fresh_2018 = preprocessing2018(fresh_2018, fresh_dict)
fresh_2021 = preprocessing2021(fresh_2021)

data1 = pd.concat([eat_2018, eat_2021], axis = 0)
data2 = pd.concat([cook_2018, cook_2021], axis = 0)
data3 = pd.concat([fresh_2018, fresh_2021], axis = 0)

In [ ]:
data1 = pd.concat([eat_2018, eat_2021], axis = 0)

## 즉석조리식품

In [ ]:
cook_2018 = preprocessing2018(cook_2018, cook_dict)
cook_2021 = preprocessing2021(cook_2021)

In [ ]:
cook_2018.iloc[0].unique()

array(['특성별(1)', '특성별(2)', '밥류', '면류', '국류', '찌개/탕류', '죽/스프류', '육류',
       '수산물', '만두/피자류', '소스류', '앙념류'], dtype=object)

In [ ]:
data2 = pd.concat([cook_2018, cook_2021], axis = 0)

# 신선편의식품

In [ ]:
fresh_2018 = preprocessing2018(fresh_2018, fresh_dict)
fresh_2021 = preprocessing2021(fresh_2021)

In [ ]:
data3 = pd.concat([fresh_2018, fresh_2021], axis = 0)

# 최종 concat

In [ ]:
data4 = pd.concat([data1, data2])

In [ ]:
data5 = pd.concat([data4, data3])

# 최종 데이터 to_csv

In [ ]:
total_df.to_csv("/content/drive/MyDrive/Team Space/data/dataset.csv")

# 최종 데이터셋

In [ ]:
# 중복 확인
total_df.drop_duplicates()

,항목,소계,수도권,충청권,호남권,대경권,동남권,강원권,동,읍면,...,여성,20대,30대,40대,50대,60대이상,200만원미만,200~400만원미만,400~600만원미만,600만원이상
2018,즉석섭취반찬류_없음,33.8,19.5,68.5,52.2,52.5,27.4,44.5,30.0,50.5,...,29.8,20.3,18.0,25.3,35.7,56.8,57.9,34.6,27.9,16.8
2018,즉석섭취반찬류_있음,66.2,80.5,31.5,47.8,47.5,72.6,55.5,70.0,49.5,...,70.2,79.7,82.0,74.7,64.3,43.2,42.1,65.4,72.1,83.2
2018,즉석섭취반찬류_매우감소,6.0,3.1,12.5,10.3,7.8,10.9,10.9,4.5,15.4,...,3.9,6.5,5.9,4.4,7.2,7.0,9.8,6.8,6.4,0.0
2018,즉석섭취반찬류_약간감소,16.8,16.3,18.5,19.9,16.6,17.5,11.5,15.1,27.3,...,11.9,10.7,17.7,14.1,14.2,28.3,21.7,17.3,17.1,10.9
2018,즉석섭취반찬류_비슷,55.2,57.9,52.8,55.4,55.0,45.7,60.2,56.4,47.7,...,61.1,48.1,49.6,59.5,57.7,55.2,50.1,53.6,53.8,67.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022,신선편의과일_약간감소,21.8,23.1,16.3,17.1,13.7,28.6,15.0,22.5,17.6,...,21.7,19.7,19.3,24.0,23.2,21.1,19.0,18.9,22.5,27.3
2022,신선편의과일_비슷,54.5,51.5,64.0,51.3,75.3,50.6,60.1,52.9,63.0,...,57.1,50.6,56.2,53.0,55.3,54.1,48.5,56.5,58.6,42.9
2022,신선편의과일_약간증가,10.4,11.4,6.1,14.3,6.9,10.6,2.1,11.0,7.7,...,8.6,13.1,11.4,10.1,9.3,10.4,13.6,9.9,8.7,14.5
2022,신선편의과일_매우증가,0.9,1.4,0.0,1.5,0.0,0.0,0.0,0.9,1.0,...,0.5,3.1,0.0,1.0,1.2,1.0,1.1,0.7,0.6,2.1


In [ ]:
len(total_df["항목"].unique())

112

In [ ]:
# total row -> 628 -> 필요없는 row 2개씩 6개 파일 = 12 -> 616
# 컬럼은 필요한 컬럼만 가져왔기 때문에 25
total_df.shape

(616, 25)

In [ ]:
# info 확인
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 616 entries, 2018 to 2022
Data columns (total 25 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   항목           616 non-null    object
 1   소계           616 non-null    object
 2   수도권          616 non-null    object
 3   충청권          616 non-null    object
 4   호남권          616 non-null    object
 5   대경권          616 non-null    object
 6   동남권          616 non-null    object
 7   강원권          616 non-null    object
 8   동            616 non-null    object
 9   읍면           616 non-null    object
 10  1인           616 non-null    object
 11  2인           616 non-null    object
 12  3인           616 non-null    object
 13  4인이상         616 non-null    object
 14  남성           616 non-null    object
 15  여성           616 non-null    object
 16  20대          616 non-null    object
 17  30대          616 non-null    object
 18  40대          616 non-null    object
 19  50대          616 non-null    o

In [ ]:
# null값 확인
total_df.isnull().sum()

항목             0
소계             0
수도권            0
충청권            0
호남권            0
대경권            0
동남권            0
강원권            0
동              0
읍면             0
1인             0
2인             0
3인             0
4인이상           0
남성             0
여성             0
20대            0
30대            0
40대            0
50대            0
60대이상          0
200만원미만        0
200~400만원미만    0
400~600만원미만    0
600만원이상        0
dtype: int64

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/techit/Team Space/data/간편식_품목별_구입경험_변화.csv")
df

,year,항목,소계,수도권,충청권,호남권,대경권,동남권,강원권,1인,...,여성,20대,30대,40대,50대,60대이상,200만원미만,200~400만원미만,400~600만원미만,600만원이상
0,2018,즉석밥류_없음,23.10,13.70,33.70,38.80,48.60,18.00,21.10,16.10,...,19.00,4.80,8.80,14.60,26.60,44.70,41.30,21.60,20.10,13.70
1,2018,즉석밥류_있음,76.90,86.30,66.30,61.20,51.40,82.00,78.90,83.90,...,81.00,95.20,91.20,85.40,73.40,55.30,58.70,78.40,79.90,86.30
2,2018,즉석밥류_매우감소,4.00,1.10,4.90,6.60,9.40,9.00,6.40,3.90,...,2.90,1.20,1.20,3.30,5.80,7.30,6.20,4.20,3.80,2.10
3,2018,즉석밥류_약간감소,10.60,9.70,11.80,15.70,11.30,10.70,6.80,8.60,...,8.30,10.40,8.00,8.30,10.20,18.50,12.00,12.60,8.60,8.10
4,2018,즉석밥류_비슷,59.10,61.30,66.30,65.00,58.90,44.70,60.30,55.30,...,61.10,48.90,53.70,57.00,64.40,67.30,63.70,56.00,59.90,63.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,2022,소스/양념류_약간감소,19.20,21.30,15.90,20.70,8.00,21.00,12.20,14.60,...,20.80,11.00,18.30,22.00,21.70,16.50,10.90,19.00,20.10,21.30
492,2022,소스/양념류_약간증가,15.60,18.70,14.40,19.20,4.70,10.20,11.90,16.40,...,13.80,19.60,17.10,16.00,14.40,14.20,11.20,17.10,15.40,14.70
493,2022,소스/양념류_없음,11.60,10.60,9.20,9.50,22.70,10.10,14.50,14.10,...,11.60,9.50,8.00,7.70,10.00,18.90,27.50,11.40,9.00,8.50
494,2022,소스/양념류_있음,88.40,89.40,90.80,90.50,77.30,89.90,85.50,85.90,...,88.40,90.50,92.00,92.30,90.00,81.10,72.50,88.60,91.00,91.50
